# Set Up

In [1]:
import pandas as pd 
import numpy  as np
import matplotlib.pyplot as plt
import os 
import seaborn as sns

# evaluate multinomial logistic regression model
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OrdinalEncoder
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from lime import lime_tabular
#MCA
import prince
from fanalysis.mca import MCA
from fanalysis.ca import CA

import scipy.stats as st

%matplotlib inline
%pylab inline

pd.options.display.max_columns = None

Populating the interactive namespace from numpy and matplotlib


In [2]:
def analyse_profil(D):
    # D doit être un crosstab avec la même variable 
    #calcul des totaux en ligne
    tot_lig = numpy.sum(D.values,axis=1)

    #calcul des totaux en colonne
    tot_col = numpy.sum(D.values,axis=0)
    #profils lignes
    prof_lig = numpy.apply_along_axis(arr=D.values,axis=1,func1d=lambda x:x/numpy.sum(x))




    #calul du profil marginal corresp. – ligne grisée de Figure 100
    prof_marg_lig = tot_col/numpy.sum(tot_col)
    #distance entre paires de modalités lignes
    distPairesLig = numpy.zeros(shape=(prof_lig.shape[0],prof_lig.shape[0]))
    #double boucle
    for i in range(prof_lig.shape[0]-1):
         for j in range(i+1,prof_lig.shape[0]):
            distPairesLig[i,j] = numpy.sum((prof_lig[i,:]-prof_lig[j,:])**2/prof_marg_lig)
    #distPairesLig[j,i] = distPairesLig[i,j]

    #affichage


    sns.heatmap(distPairesLig,vmin=0,vmax=numpy.max(distPairesLig),linewidth=0.1,cmap='Blues',
                xticklabels=D.index,yticklabels=D.index)
    #à revoir la forme du dataframe  
    print(pd.DataFrame(distPairesLig,index=D.index,columns=D.index))

In [3]:
def chi2(cont):
    #def nos deux variable 
    # créer tableau de cont 
    
        st_chi2, st_p, st_dof, st_exp = st.chi2_contingency(cont)

    # interpret p-value
        alpha = 0.05
        print(" Pearson’s chi-squared test for independence")
        print('       Chi2=%.3f, st_p=%.3f' % (st_chi2, st_p))
        if st_p <= alpha:
            print("    ")
            print('Les variables sont dépendentes, la liaison est significative entre les deux variables')
            #Le test conduit au rejet de l’hypothèse nulle (p-value < 5%).
            #Manifestement, les variables var1 et var2 sont fortement liées.
            print("")

        else:
            print('Les variables sont indépendentes')

In [4]:
def co_a(df_1,df_2): 
    ca = prince.CA(
    n_components=3,
    n_iter=3,
    copy=True,
    check_input=True,
    engine="auto",
    random_state=42)
    global X
    X = pd.crosstab(df_1,df_2 ,dropna=False,margins=False, normalize=False)
    ca = ca.fit(X)
    ax = ca.plot_coordinates(X=X, 
                        ax=None,
                        figsize=(10, 9),
                        x_component=0, 
                        y_component=1, 
                        show_row_labels=True, 
                        show_col_labels=True)    
    
    #barplot = X.plot.bar(stacked=True,)
    
    #effectifs totaux
    tot_lig = np.sum(X.values,axis=1)
    tot_col = np.sum(X.values,axis=0)
    n = np.sum(X.values)
    #tableau sous indépendance
    E = np.dot(np.reshape(tot_lig,(len(tot_lig),1)),
               np.reshape(tot_col,(1,len(tot_col))))/n  #len(tot_col)
    #print(E)

    #tableau des indices d'attraction et répulsion
    IAR = X.values/E

    #pour rappel - les indices observés
    #Si l’indice est supérieur à 1, nous avons une attraction ; 
    #s’il est inférieur, nous avons une répulsion entre les modalités.

    #pd.DataFrame(IAR,index=D.index,columns=D.columns)

    #approximation des indices d'attraction-répulsion
    afc = CA(row_labels=X.index,col_labels=X.columns)
    afc.fit(X.values)
    estIAR = 1+np.dot(np.reshape(afc.row_coord_[:,0],(X.shape[0],1)),
                      np.reshape(afc.col_coord_[:,0],
                     (1,X.shape[1])))/np.sqrt(afc.eig_[0][0])
    return pd.DataFrame(estIAR,index=X.index,columns=X.columns)
    
     #  à remplacer  pour interprétation
    

In [5]:
def rep(X):
    tot_lig = np.sum(X.values,axis=1)
    tot_col = np.sum(X.values,axis=0)
#profils lignes
    prof_lig = np.apply_along_axis(arr=X.values, axis=1,func1d=lambda x:x/np.sum(x))

#rep graph
    somme = np.zeros(shape=(prof_lig.shape[0]))
    for i in range(prof_lig.shape[1]):
        plt.barh(range(prof_lig.shape[0]),prof_lig[:,i],left=somme)
        somme = somme + prof_lig[:,i]
    plt.yticks(range(prof_lig.shape[0]),X.index)
    plt.legend(X.columns)
    plt.show()


In [6]:
def etude_prof(D):
    #calcul des totaux en ligne
    tot_lig = np.sum(D.values,axis=1)

    #calcul des totaux en colonne
    tot_col = np.sum(D.values,axis=0)

    #profils lignes
    prof_lig = np.apply_along_axis(arr=D.values,axis=1,func1d=lambda x:x/numpy.sum(x))

    #calul du profil marginal corresp. – ligne grisée de Figure 100 
    prof_marg_lig = tot_col/np.sum(tot_col)

    #distance entre paires de modalités lignes
    distPairesLig = numpy.zeros(shape=(prof_lig.shape[0],prof_lig.shape[0]))


    #double boucle
    for i in range(prof_lig.shape[0]-1):
        for j in range(i+1,prof_lig.shape[0]):
            distPairesLig[i,j] = np.sum((prof_lig[i,:]-prof_lig[j,:])**2/prof_marg_lig) 
            #distPairesLig[j,i] = distPairesLig[i,j]


#affichage
    print(pd.DataFrame(distPairesLig,index=D.index,columns=D.index))


#affichage sous forme de heatmap
    sns.heatmap(distPairesLig,vmin=0,
                vmax=numpy.max(distPairesLig),
                linewidth=0.1,cmap= 'Blues',
                xticklabels=D.index,
                yticklabels=D.index)

In [7]:
def mca(df):
    #from fanalysis.mca import MCA
    X = df.iloc[:, 0:len(df.columns)].values

    my_mca = MCA(row_labels=df.index.values, 
             var_labels=df.columns.values[0:6])

    my_mca.fit(X)

    df_rows = my_mca.row_topandas()
    df_cols = my_mca.col_topandas()

    my_mca.mapping(num_x_axis=1, num_y_axis=2, figsize=(20, 18))

    my_mca.mapping_col(num_x_axis=1, num_y_axis=2, figsize=(20, 18))
    
    # Classement des points colonnes en fonction de leur contribution au 1er axe
    my_mca.plot_col_contrib(num_axis=1)
    
    # Classement des points colonnes en fonction de leur cos2 sur le 1er axe
    my_mca.plot_col_cos2(num_axis=1)

In [8]:
def mca_v2(df):
    mca = prince.MCA(
        n_components=2,
        n_iter=3,
        copy=True,
        check_input=True,
        engine='auto',
        random_state=42
        )
    df_Sat_mca = mca.fit(df_Sat)

    ax = df_Sat_mca.plot_coordinates(
        X=df_Sat,
        ax=None,
        figsize=(20, 18),
        show_row_points=False,
        row_points_size=0,
        show_row_labels=False,
        show_column_points=True,
        column_points_size=30,
        show_column_labels=True,
        legend_n_cols=1
               ).legend(loc='center left', bbox_to_anchor=(1, 1))